In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-01-14 23:48:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
connected. to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-01-14 23:48:53 (57.3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [11]:
import minsearch

In [12]:
import json

In [13]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)
    

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']= course_dict['course']
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [17]:
index.fit(documents)

In [18]:
ques = "the course has already started, can I still enroll?"

In [19]:
#to give more importance to one text field as compared to others

boost = {'question':3.0, 'section':0.5}
results = index.search(
    query=ques,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results=5
)

In [20]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [21]:
###using openai
from openai import OpenAI

In [22]:
client = OpenAI()

In [23]:
ques

'the course has already started, can I still enroll?'

In [31]:
prompt

"You're a course teaching assistant. \nAnswer the QUESTION based on the facts from the CONTEXT from the FAQ database. \nUse only the facts from the context when answering the QUESTION. \nIf the context does not contain the answer, output none.\nQUESTION: the course has already started, can I still enroll?\nCONTEXT: section : General course-related questions\nquestion:Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection : General course-related questions\nquestion:Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you ca

### Clear Pipeline for RAG Application

In [34]:
def search(query):
    #to give more importance to one text field as compared to others
    boost = {'question':3.0, 'section':0.5}
    results = index.search(
        query=ques,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results=5
    )
    return results

In [37]:
def build_prompt(query,search_results):
    prompt_template="""
    You're a course teaching assistant. 
    Answer the QUESTION based on the facts from the CONTEXT from the FAQ database. 
    Use only the facts from the context when answering the QUESTION. 
    If the context does not contain the answer, output none.
    QUESTION: {question}
    CONTEXT: {context}
    """.strip()
    context = ""
    for doc in results:
        context = context + f"section : {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question = ques, context = context).strip()
    return prompt

In [42]:
def llm(prompt):
    response = client.chat.completions.create(
        model ='gpt-3.5-turbo',
        messages=[{"role":"user","content": prompt}]
    )
    return response.choices[0].message.content

In [50]:
def rag(query):
    query = "The course already started, how do I enroll?"
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [51]:
rag(query)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}